In [4]:
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
import os

In [2]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [5]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database='neo4j'
)

In [6]:
cypher = """
   MATCH (t:Title)-[:RELATED_TO]->(top:Topic) 
  RETURN t.title, t.source, top LIMIT 10

  """

result = kg.query(cypher)
result

[{'t.title': 'Machine-learning',
  't.source': 'https://en.wikipedia.org/wiki/Machine_learning',
  'top': {'name': 'machine-learning'}},
 {'t.title': 'Machine-learning',
  't.source': 'https://en.wikipedia.org/wiki/Machine_learning',
  'top': {'name': 'neural-networks'}},
 {'t.title': 'Machine-learning',
  't.source': 'https://en.wikipedia.org/wiki/Machine_learning',
  'top': {'name': 'artificial-intelligence'}},
 {'t.title': 'Quantum-machine-learning',
  't.source': 'https://en.wikipedia.org/wiki/Quantum_machine_learning',
  'top': {'name': 'classical-machine-learning-applied-to-quantum-experiments'}},
 {'t.title': 'Quantum-machine-learning',
  't.source': 'https://en.wikipedia.org/wiki/Quantum_machine_learning',
  'top': {'name': 'quantum-machine-learning'}},
 {'t.title': 'Quantum-machine-learning',
  't.source': 'https://en.wikipedia.org/wiki/Quantum_machine_learning',
  'top': {'name': 'quantum-computing'}},
 {'t.title': 'Neural-network-(machine-learning)',
  't.source': 'https://e

## Graphwidget

In [7]:
from yfiles_jupyter_graphs import GraphWidget

In [8]:
w = GraphWidget()

# Create dictionaries to store unique nodes and edges
nodes_dict = {}
edges = []
node_id = 0
edge_id = 0

# Process each record to populate nodes and edges
for record in result:
    title = record['t.title']
    topic_name = record['top']['name']
    
    # Check if the title node already exists; if not, create it
    if title not in nodes_dict:
        nodes_dict[title] = node_id
        w.nodes.append({
            "id": node_id,
            "properties": {
                "label": title,
                "source": record['t.source']
            }
        })
        node_id += 1
    
    # Check if the topic node already exists; if not, create it
    if topic_name not in nodes_dict:
        nodes_dict[topic_name] = node_id
        w.nodes.append({
            "id": node_id,
            "properties": {"label": topic_name}
        })
        node_id += 1
    
    # Create an edge between title and topic
    w.edges.append({
        "id": edge_id,
        "start": nodes_dict[title],
        "end": nodes_dict[topic_name],
        "properties": {"label": "RELATED_TO"}
    })
    edge_id += 1

w.directed = True

In [9]:
display(w)

GraphWidget(layout=Layout(height='630px', width='100%'))

## Neo4J Graph

In [13]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD))
session = driver.session()

In [14]:
def showGraph(cypher: str):
    widget = GraphWidget(graph = session.run(cypher).graph()) 
    display(widget)
    return widget

In [23]:
cypher = """
   MATCH (n)
  RETURN n
  """

In [24]:
w = showGraph(cypher)

GraphWidget(layout=Layout(height='800px', width='100%'))

In [28]:
cypher = """
   MATCH (t:Title)-[r:RELATED_TO]->(top:Topic) 
  RETURN t, r, top
  """

w2 = GraphWidget(graph = session.run(cypher).graph())

def custom_node_label_mapping(index, node):
    """let the label be the name or the title"""
    properties = node.get('properties', {})
    return properties.get('title', properties.get('name', 'no label'));
w2.node_label_mapping = custom_node_label_mapping

def custom_relationship_label_mapping(index, node):
    """let the label be the role"""
    properties = node.get('properties', {})
    return properties.get('RELATED_TO', ['no role'])[0];
w2.edge_label_mapping = custom_relationship_label_mapping

display(w2)

GraphWidget(layout=Layout(height='800px', width='100%'))